In [ ]:
import cv2
import numpy as np
import mediapipe as mp

# إعداد mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# ضع مسار الفيديو هنا
video_path = "id2_jab_2.mp4"
cap = cv2.VideoCapture(video_path)

# قراءة أول فريم لمعرفة الأبعاد الأصلية بعد التدوير
ret, first_frame = cap.read()
if not ret:
    raise Exception("❌ لم يتمكن من قراءة الفيديو!")

# تدوير أول فريم لمعرفة الأبعاد الجديدة
first_frame_rotated = cv2.rotate(first_frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
frame_height, frame_width = first_frame_rotated.shape[:2]

# إعداد حفظ الفيديو بعد التدوير
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter("classified_output_fixed.mp4", fourcc, fps, (frame_width, frame_height))

# دالة تصنيف بسيطة
def classify_punch(wrist, elbow):
    dx = wrist[0] - elbow[0]
    dy = wrist[1] - elbow[1]
    angle = np.degrees(np.arctan2(dy, dx))

    if -30 <= angle <= 30:
        return "jab"
    elif 60 <= abs(angle) <= 120:
        return "hook"
    elif angle > 120 or angle < -120:
        return "uper"
    else:
        return "non"

# إعادة موضع الفيديو لبداية التشغيل
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# المعالجة
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # تدوير الإطار 90 درجة عكس عقارب الساعة ليصبح الفيديو بالطول
        frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        label = "non"
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            right_wrist = (landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y)
            right_elbow = (landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y)

            label = classify_punch(right_wrist, right_elbow)
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.putText(frame, f"Punch: {label}", (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)
        out.write(frame)

cap.release()
out.release()